In [264]:
import pygmsh
import meshio
import pygalmesh
import hoomd
import hoomd.md
import numpy as np
import copy

# Make a basic mesh

In [342]:
# https://github.com/nschloe/pygmsh/issues/262
#with pygmsh.geo.Geometry() as geom:
#    geom.add_circle([0,0],10,mesh_size=20)
#    mesh = geom.generate_mesh()

In [343]:
with pygmsh.occ.Geometry() as geom:
    geom.characteristic_length_max = 100
    geom.add_disk([0.0, 0.0], 10.0,mesh_size=100),

    mesh = geom.generate_mesh()

In [309]:
mesh

<meshio mesh object>
  Number of points: 8
  Number of cells:
    line: 7
    triangle: 7
    vertex: 1

Get all the bonds, in an ordered, unique, list

In [310]:
triangles=mesh.cells[1].data
x = [[[triangle[0],triangle[1]],[triangle[0],triangle[2]],[triangle[1],triangle[2]] ]   for triangle in triangles]
flattenedx = [val for sublist in x for val in sublist]
orderedx = [[x[0],x[1]] if x[0]<x[1] else [x[1],x[0]] for x in flattenedx]
bonds = []

bonds = []
for elem in orderedx:
    if elem not in bonds:
        bonds.append(elem)

# sort the edge bonds
edgebonds = [[x[0],x[1]] if x[0]<x[1] else [x[1],x[0]] for x in mesh.cells[0].data]
interiorbonds=[x for x in bonds if x not in edgebonds]

In [311]:
Outputmesh=copy.deepcopy(mesh)

In [312]:
mesh.write("Disk.vtk")

 ## Initialising the simulation 

In [375]:
hoomd.context.initialize("");

Read in the list of points and bonds

In [376]:
Npts=len(mesh.points);
snapshot = hoomd.data.make_snapshot(N=Npts
                                    ,box=hoomd.data.boxdim(Lx=200, Ly=200,dimensions=2)
                                    ,particle_types=['A']
                                    ,bond_types=['Spring']);
snapshot.particles.position[:] = mesh.points;
snapshot.particles.typeid[0:N]=0

In [377]:
Nbonds=len(bonds);
snapshot.bonds.resize(Nbonds);
snapshot.bonds.group[:] = bonds;

In [378]:
system= hoomd.init.read_snapshot(snapshot);

notice(2): Group "all" created containing 8 particles


Make all the bonds hookean springs

In [379]:
harmonic = hoomd.md.bond.harmonic();
harmonic.bond_coeff.set('Spring', k=0.1, r0=8);

Define the integrator. In this case, a langevin dynamics

In [380]:
hoomd.md.integrate.mode_standard(dt=0.1);
all = hoomd.group.all();
integrator = hoomd.md.integrate.langevin(group=all,kT=0,seed=0,dscale=0.1);

notice(2): integrate.langevin/bd is determining gamma from particle diameters


Define a callback, which we want run periodically

In [381]:
class WritePositions:
    def __init__(self, system):
        self.system = system;
    def __call__(self, timestep):
        snap = self.system.take_snapshot();
        Outputmesh.points = snap.particles.position
        Outputmesh.write("Output"+str(timestep)+".vtk")
        
hoomd.analyze.callback(callback=WritePositions(system), period=10);

# Run the simulation

In [382]:
hoomd.run(10000);

** starting run **
Time 00:00:03 | Step 10000 / 10000 | TPS 15253.8 | ETA 00:00:00
Average TPS: 15237.2
---------
** run complete **


In [361]:
dir(snapshot)
dir(snapshot.particles)
snapshot.particles.diameter

array([1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)